In [1]:
import torch
import pickle
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy
from tqdm import tqdm


from model import LR
from data import FairnessDataset, SyntheticDataset, GermanDataset, IncomeDataset
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *
from ei_model_test import EIModel, fair_batch_proxy, covariance_proxy

# warnings.filterwarnings('ignore')

In [11]:
# dataset = SyntheticDataset(seed=1)
# dataset = GermanDataset(seed=1)
dataset = IncomeDataset(seed=1)

In [12]:
torch.manual_seed(0)
train_tensors, val_tensors, test_tensors = dataset.tensor(z_blind=False)
train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)

model = LR(train_dataset.X.shape[1])

tau = 0.5
delta = 0.5
lamb = 1.
alpha = 0.
effort = Optimal_Effort(delta)
proxy = fair_batch_proxy
lr = 1e-2
n_epochs = 100

# EI Model
ei_model = EIModel(deepcopy(model), proxy, effort, tau)
ei_model.train(train_dataset, lamb, alpha, lr, n_epochs)
for module in ei_model.model.layers:
    if hasattr(module, 'weight'):
        weights_0 = module.weight.data
    if hasattr(module, 'bias'):
        bias_0 = module.bias.data
theta_0 = torch.cat((weights_0[0], bias_0), 0)
print(theta_0)

# # REI Model
# rei_model = EIModel(deepcopy(model), proxy, effort, tau)
# rei_model.train(train_dataset, lamb, alpha, n_epochs=200, abstol=1e-7)
# for module in rei_model.model.layers:
#     if hasattr(module, 'weight'):
#         weights_r_0 = module.weight.data
#     if hasattr(module, 'bias'):
#         bias_r_0 = module.bias.data
# theta_0_r = torch.cat((weights_r_0[0], bias_r_0), 0)

# print(theta_0_r)

Training [alpha=0.00; lambda=1.00; delta=0.50]: 100%|██████████| 100/100 [02:22<00:00,  1.43s/epochs]

tensor([-5.0768e-02,  6.1250e-01, -2.0084e-02,  2.5392e-01,  1.5941e-06,
        -2.0984e-08,  1.8928e-38, -4.2040e-13,  2.4220e-02,  4.4699e-41,
        -1.0041e-04, -1.3343e-01,  1.4816e-06, -2.1034e-07,  1.7032e-03,
         4.2090e-01,  3.5859e-04,  1.8606e-11, -4.1629e-08,  1.5001e-10,
        -7.3839e-07,  2.7593e-40,  1.6990e-06, -4.1140e-04,  2.3347e-08,
         1.2771e-08,  7.0285e-05, -1.1125e-03, -1.7678e-11,  1.1844e-11,
         7.3936e-05, -2.3085e-10, -2.3214e-01, -9.6004e-13,  2.4126e-07,
         1.4484e-04, -3.9083e-09, -2.1511e-41,  6.3041e-09, -2.1257e-08,
        -1.5700e-11, -1.0670e-04, -2.7526e-12,  1.7122e-07, -8.3404e-08,
        -9.8763e-07,  2.2219e-10, -9.1111e-06,  1.9132e-01, -3.4500e-12,
        -3.4430e-09,  8.2701e-12, -2.2346e-11, -2.1782e-10,  2.4815e-07,
        -7.8131e-09,  2.1014e-08,  2.3217e-09,  1.0620e-07, -1.2352e-06,
         6.6183e-10, -6.0239e-14, -1.3673e-11,  7.3062e-40,  1.8677e-10,
         2.2926e-07,  2.2274e-10,  2.2041e-08,  9.2

In [19]:
ei_model.model(test_dataset.X).reshape(-1)

tensor([0.9999, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
       grad_fn=<ViewBackward0>)

In [13]:
lf_n = 48
alpha = alpha
print(f'Train Parameters')
print('-'*lf_n)
print(f'Dataset                   |   {dataset.__class__.__name__}')
print(f'Proxy                     |   {proxy.__name__}')
print(f'Effort                    |   {effort.__class__.__name__}')
print(f'alpha                     |   {alpha}')
print(f'lambda                    |   {lamb}')
print(f'delta                     |   {delta}')
print(f'GD Epochs                 |   {n_epochs}')
print(f'GD lr                     |   {lr}')
print(f'PGA Epochs                |   until convergence')
print('-'*lf_n)
print('-'*lf_n)
print('Evaluation Results')
print('-'*38)
for alpha_eval in [0., 0.1, 0.5, 1.5, 5.0]:
    Y_hat, Y_hat_max, fair_loss = ei_model.predict(test_dataset, alpha_eval, 1e-7)
    accuracy, ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
    for module in ei_model.model_adv.layers:
        if hasattr(module, 'weight'):
            weights_adv = module.weight.data
        if hasattr(module, 'bias'):
            bias_adv = module.bias.data
    theta_adv = torch.cat((weights_adv[0], bias_adv), 0)
    alphas = (theta_adv-theta_0).abs()

    # Y_hat_r, Y_hat_max_r, fair_loss_r = rei_model.predict(test_dataset, alpha, 1e-7)
    # accuracy_r, rei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat_r, Y_hat_max_r, tau)
    # for module in ei_model.model_adv.layers:
    #     if hasattr(module, 'weight'):
    #         weights_adv_r = module.weight.data
    #     if hasattr(module, 'bias'):
    #         bias_adv_r = module.bias.data
    # theta_adv_r = torch.cat((weights_adv_r[0], bias_adv_r), 0)
    # alphas_r = (theta_adv_r-theta_0_r).abs()

    print(f'alpha                     |   {alpha_eval}')
    print(f'Accuracy                  |   {accuracy:.5f}')
    # print(f'Accuracy (Robust)         |   {accuracy_r:.5f}')
    print(f'Fairness Loss             |   {fair_loss:.5f}')
    # print(f'Fairness Loss (Robust)    |   {fair_loss_r:.5f}')
    print(f'EI Disparity              |   {ei_disparity:.5f}')
    # print(f'EI Disparity (Robust)     |   {rei_disparity:.5f}')
    # print(f'alphas                    |   {np.round(alphas, 4)}')
    # print(f'alphas (Robust)           |   {np.round(alphas_r, 4)}')
    # print(theta_0)
    # print(theta_adv)
    # print(theta_adv_r)
    print('-'*38)

Train Parameters
------------------------------------------------
Dataset                   |   IncomeDataset
Proxy                     |   fair_batch_proxy
Effort                    |   Optimal_Effort
alpha                     |   0.0
lambda                    |   1.0
delta                     |   0.5
GD Epochs                 |   100
GD lr                     |   0.01
PGA Epochs                |   until convergence
------------------------------------------------
------------------------------------------------
Evaluation Results
--------------------------------------


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn